In [ ]:
# This is the code we used to infer the bacterial communities of our bees
# Starting from our cleaned total metagenome reads, it...
    # Runs kraken2 to map read khmers to the PlusPF_db database
    # puts the kraken2 output through braken to make .biom file for use in phyloseq and R

#!/bin/bash
#SBATCH --job-name=kraken_db_%j
#SBATCH --nodes=1
#SBATCH -t 3:00:00
#SBATCH --ntasks=24
#SBATCH --output=/home/dsbard/bee_phage/dls/total_metagenomes/kraken/log/kraken_db_%j.out
#SBATCH --partition=bigmemm
#SBATCH --mem 250GB

conda activate kraken2

kraken2 --db /home/dsbard/bee_phage/dls/total_metagenomes/kraken/PlusPF_db \
--threads 24 \
--paired /home/dsbard/bee_phage/dls/total_metagenomes/decontam/$1 \
/home/dsbard/bee_phage/dls/total_metagenomes/decontam/${1%%R1*}R2.fastq.gz \
--output /home/dsbard/bee_phage/dls/total_metagenomes/kraken/out/PlusPF/${1%%_host*}.kraken \
--report /home/dsbard/bee_phage/dls/total_metagenomes/kraken/out/PlusPF/${1%%_host*}.report

# We then use bracken to estimate abundance (using default parameters)
conda activate bracken
for f in *.report
do
bracken -d home/dsbard/bee_phage/dls/total_metagenomes/kraken/PlusPF_db \
-i $f -o ./bracken/${f%%.report}.bracken
done

# lastly, use kraken-biom to collect data into a .biom file
cd ./bracken
kraken-biom *.report --fmt json -o bee_S.biom